In [1]:
sc

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
8,application_1579235851612_0009,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-8>

In [2]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

sc = SparkContext.getOrCreate()
ss = SparkSession.builder.master('yarn').config("spark.dynamicAllocation.enabled", True).getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
LA_left_lag30 = ss.read.parquet("s3a://dc-group2/LA_left_lag30")
LA_left_lag30 = LA_left_lag30.where('prev30 is not null')
lag30 = LA_left_lag30.drop('State_Name', 'County_Name','Date')

from pyspark.ml.feature import VectorAssembler
va = VectorAssembler(outputCol="features", inputCols=lag30.drop("label").columns) 
lag30_va = va.transform(lag30).select("features", "label")

lag30_va_set = lag30_va.randomSplit([0.8, 0.2])
lag30_train = lag30_va_set[0].cache()
lag30_test = lag30_va_set[1].cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
from pyspark.ml.regression import RandomForestRegressor
rf = RandomForestRegressor()
rfmodel = rf.fit(lag30_train)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
rfpred = rfmodel.transform(lag30_test)

from pyspark.ml.evaluation import RegressionEvaluator
evaluator1 = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="r2")
evaluator2 = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse") 
evaluator3 = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="mae")
# supports "rmse" (default), "mse", "r2", and "mae"
r2 = evaluator1.evaluate(rfpred)
rmse = evaluator2.evaluate(rfpred)
mae = evaluator3.evaluate(rfpred)
print("R2 on test data = %g" % r2)
print("rmse on test data = %g" % rmse)
print("mae on test data = %g" % mae)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

R2 on test data = 0.704837
rmse on test data = 33.7654
mae on test data = 25.3047